# Calculate grit for different Cell Profiler feature groups

* Per compartment (Cells, Cytoplasm, Nuclei)
* Per compartment feature group (Cells x AreaShape, Cells x Correlation, Nuclei x Texture, etc.)
* Per channel (DNA, RNA, AGP, ER, Mito)
    * Use all features that include any information from one of these channels

In [2]:
import pathlib
import numpy as np
import pandas as pd

from pycytominer.cyto_utils import infer_cp_features

from cytominer_eval import evaluate_metrics

In [3]:
compartments = ["Cells", "Cytoplasm", "Nuclei"]

In [10]:
# Load Cell Health data
data_file = pathlib.Path("../../0.download-data/data/cell-health/profiles/cell_health_profiles_merged_wholeplate_normalized_featureselected.tsv.gz")

df = pd.read_csv(data_file, sep="\t")

print(df.shape)
df.head(2)

(3456, 513)


,Metadata_Plate,Metadata_Well,Metadata_WellCol,Metadata_WellRow,Metadata_cell_line,Metadata_gene_name,Metadata_pert_name,Cells_AreaShape_Compactness,Cells_AreaShape_Extent,Cells_AreaShape_FormFactor,...,Nuclei_Texture_SumAverage_DNA_20_0,Nuclei_Texture_SumAverage_ER_20_0,Nuclei_Texture_SumAverage_Mito_5_0,Nuclei_Texture_SumAverage_RNA_5_0,Nuclei_Texture_SumEntropy_Mito_5_0,Nuclei_Texture_SumEntropy_RNA_20_0,Nuclei_Texture_SumEntropy_RNA_5_0,Nuclei_Texture_Variance_ER_20_0,Nuclei_Texture_Variance_Mito_20_0,Nuclei_Texture_Variance_RNA_10_0
0,SQ00014612,A01,1,A,A549,EMPTY,EMPTY,-0.378191,-0.030782,2.089629,...,-2.46262,2.556333,0.635753,2.262901,1.020190,-1.051497,1.690149,1.211959,0.904173,0.283193
1,SQ00014612,A02,2,A,A549,MCL1,MCL1-5,0.739037,-0.466950,2.245620,...,-2.52732,2.099596,0.387399,1.606377,0.619004,-1.123289,1.319333,1.126989,1.149692,0.538925


In [11]:
# Define cell health constants
barcode_col = "Metadata_pert_name"
gene_col = "Metadata_gene_name"

replicate_group_grit = {"profile_col": barcode_col, "replicate_group_col": gene_col}

control_group_cut = ["Chr2", "Luc", "LacZ"]

control_barcodes = (
    df.loc[
        df[replicate_group_grit["replicate_group_col"]].isin(control_group_cut),
        replicate_group_grit["profile_col"],
    ]
    .unique()
    .tolist()
)

control_barcodes

['Chr2-1',
 'Chr2-4',
 'Chr2-5',
 'Chr2-2',
 'Luc-1',
 'LacZ-3',
 'Luc-2',
 'LacZ-2',
 'Chr2-3',
 'Chr2-6']

In [5]:
pair_config = {
    "pos_sameby": {"all": ["Metadata_pert_name", "Metadata_is_control == 0"], "any": []},
    "pos_diffby": {"all": [], "any": []},
    "neg_sameby": {"all": [], "any": []},
    "neg_diffby": {"all": ["Metadata_is_control"], "any": []},
}

map_config = {
    "mean_ap": {
        "null_size": 10000,
        "groupby_columns": ["Metadata_pert_name"],
    },
    # "mp_value": {
    #     "control_perts": [1],
    #     "replicate_id": f"{gene_col}",
    #     "rescale_pca": True,
    #     "nb_permutations": 1000,
    # },
}

In [6]:
all_features = infer_cp_features(df, compartments=compartments)
meta_features = infer_cp_features(df, metadata=True)

meta_features

['Metadata_Plate',
 'Metadata_Well',
 'Metadata_WellCol',
 'Metadata_WellRow',
 'Metadata_cell_line',
 'Metadata_gene_name',
 'Metadata_pert_name']

In [7]:
map_compartment_results = []
for cell_line in df.Metadata_cell_line.unique():
    for compartment in compartments:
        compartment_features = infer_cp_features(df, compartments=compartment)

        for drop in [True, False]:
            if drop:
                subset_df = df.drop(compartment_features, axis="columns")
                dropped_or_exclusive = "dropped"
                use_features = subset_df.drop(meta_features, axis="columns").columns.tolist()
            else:
                subset_df = df.loc[:, meta_features + compartment_features]
                dropped_or_exclusive = "exclusive"
                use_features = compartment_features

            subset_df = subset_df.query("Metadata_cell_line == @cell_line").reset_index(drop=True)
            subset_df["Metadata_is_control"] = subset_df.Metadata_pert_name.isin(control_barcodes).astype(int)

            result = evaluate_metrics(
                profiles=subset_df,
                features=use_features,
                meta_features=subset_df.filter(regex="Metadata_").columns,
                replicate_groups=pair_config,
                metrics_config=map_config,
                use_copairs=True,
            )["mean_ap"]

            # result = evaluate_metrics(
            #     profiles=subset_df,
            #     features=use_features,
            #     meta_features=[barcode_col, gene_col],
            #     replicate_groups=replicate_group_grit,
            #     operation="grit",
            #     similarity_metric="pearson",
            #     grit_control_perts=control_barcodes,
            # )
            
            result = result.assign(
                cell_line=cell_line,
                barcode_control="cutting_control",
                # cor_method="pearson",
                compartment=compartment,
                channel="all",
                feature_group="all",
                num_features=len(compartment_features),
                dropped_or_exclusive=dropped_or_exclusive,
            )

            map_compartment_results.append(result)

map_compartment_results = pd.concat(map_compartment_results).reset_index(drop=True)

print(map_compartment_results.shape)
map_compartment_results.head()


Calculating distances.
Pos pairs size: 21912, Neg pairs size: 184320


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]


Calculating metric: mean_ap


/dgx1nas1/storage/data/akalinin/projects/copairs/src/copairs/compute.py:105: RuntimeWarning: invalid value encountered in divide
  ap_scores = np.add.reduceat(pr_k * rel_k_list, cutoffs) / num_pos


...


Calculating metric: mean_ap


/dgx1nas1/storage/data/akalinin/projects/copairs/src/copairs/compute.py:105: RuntimeWarning: invalid value encountered in divide
  ap_scores = np.add.reduceat(pr_k * rel_k_list, cutoffs) / num_pos


  0%|          | 0/6 [00:00<?, ?it/s]

(1962, 13)


,Metadata_gene_name,Metadata_pert_name,mean_ap,p_value,n_pos_pairs,n_total_pairs,cell_line,barcode_control,compartment,channel,feature_group,num_features,dropped_or_exclusive
0,AKT1,AKT1-1,0.243094,0.084708,11.0,203.0,HCC44,cutting_control,Cells,all,all,100,dropped
1,AKT1,AKT1-2,0.206240,0.101630,11.0,203.0,HCC44,cutting_control,Cells,all,all,100,dropped
2,ARID1B,ARID1B-1,0.122818,0.155451,11.0,203.0,HCC44,cutting_control,Cells,all,all,100,dropped
3,ARID1B,ARID1B-2,0.126775,0.125047,11.0,203.0,HCC44,cutting_control,Cells,all,all,100,dropped
4,ATF4,ATF4-1,0.334947,0.024968,11.0,203.0,HCC44,cutting_control,Cells,all,all,100,dropped


## Calculate grit for feature groups

In [8]:
feature_group_compartments = list(set(["_".join(x.split("_")[0:2]) for x in all_features]))

map_subcompartment_results = []
for cell_line in df.Metadata_cell_line.unique():
    for compartment_group in feature_group_compartments:
        compartment, feature_group = compartment_group.split("_")
        compartment_features = df.loc[:, df.columns.str.startswith(compartment_group)].columns.tolist()

        for drop in [True, False]:
            if drop:
                subset_df = df.drop(compartment_features, axis="columns")
                dropped_or_exclusive = "dropped"
                use_features = subset_df.drop(meta_features, axis="columns").columns.tolist()
            else:
                subset_df = df.loc[:, meta_features + compartment_features]
                dropped_or_exclusive = "exclusive"
                use_features = compartment_features

            subset_df = subset_df.query("Metadata_cell_line == @cell_line").reset_index(drop=True)
            subset_df["Metadata_is_control"] = subset_df.Metadata_pert_name.isin(control_barcodes).astype(int)

            result = evaluate_metrics(
                profiles=subset_df,
                features=use_features,
                meta_features=subset_df.filter(regex="Metadata_").columns,
                replicate_groups=pair_config,
                metrics_config=map_config,
                use_copairs=True,
            )["mean_ap"]

            # result = evaluate(
            #     profiles=subset_df.query("Metadata_cell_line == @cell_line"),
            #     features=use_features,
            #     meta_features=[barcode_col, gene_col],
            #     replicate_groups=replicate_group_grit,
            #     operation="grit",
            #     similarity_metric="pearson",
            #     grit_control_perts=control_barcodes,
            # )
            
            result = result.assign(
                cell_line=cell_line,
                barcode_control="cutting_control",
                # cor_method="pearson",
                compartment=compartment,
                channel="all",
                feature_group=feature_group,
                num_features=len(compartment_features),
                dropped_or_exclusive=dropped_or_exclusive,
            )

            map_subcompartment_results.append(result)

map_subcompartment_results = pd.concat(map_subcompartment_results).reset_index(drop=True)

print(map_subcompartment_results.shape)
map_subcompartment_results.head()


Calculating distances.
Pos pairs size: 21912, Neg pairs size: 184320


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]


Calculating metric: mean_ap


/dgx1nas1/storage/data/akalinin/projects/copairs/src/copairs/compute.py:105: RuntimeWarning: invalid value encountered in divide
  ap_scores = np.add.reduceat(pr_k * rel_k_list, cutoffs) / num_pos


...


Calculating metric: mean_ap


/dgx1nas1/storage/data/akalinin/projects/copairs/src/copairs/compute.py:105: RuntimeWarning: invalid value encountered in divide
  ap_scores = np.add.reduceat(pr_k * rel_k_list, cutoffs) / num_pos


  0%|          | 0/6 [00:00<?, ?it/s]

(12426, 13)


,Metadata_gene_name,Metadata_pert_name,mean_ap,p_value,n_pos_pairs,n_total_pairs,cell_line,barcode_control,compartment,channel,feature_group,num_features,dropped_or_exclusive
0,AKT1,AKT1-1,0.238305,0.124004,11.0,203.0,HCC44,cutting_control,Cells,all,Correlation,20,dropped
1,AKT1,AKT1-2,0.196297,0.148785,11.0,203.0,HCC44,cutting_control,Cells,all,Correlation,20,dropped
2,ARID1B,ARID1B-1,0.119698,0.169400,11.0,203.0,HCC44,cutting_control,Cells,all,Correlation,20,dropped
3,ARID1B,ARID1B-2,0.123932,0.145555,11.0,203.0,HCC44,cutting_control,Cells,all,Correlation,20,dropped
4,ATF4,ATF4-1,0.324306,0.023688,11.0,203.0,HCC44,cutting_control,Cells,all,Correlation,20,dropped


## Calculate grit for channels

In [17]:
channels = ["DNA", "RNA", "Mito", "AGP", "ER"]

map_channel_results = []
for cell_line in df.Metadata_cell_line.unique():
    for channel in channels:
        channel_features = df.loc[:, df.columns.str.contains(channel)].columns.tolist()

        for drop in [True, False]:
            if drop:
                subset_df = df.drop(channel_features, axis="columns")
                dropped_or_exclusive = "dropped"
                use_features = subset_df.drop(meta_features, axis="columns").columns.tolist()
            else:
                subset_df = df.loc[:, meta_features + channel_features]
                dropped_or_exclusive = "exclusive"
                use_features = channel_features

            subset_df = subset_df.query("Metadata_cell_line == @cell_line").reset_index(drop=True)
            subset_df["Metadata_is_control"] = subset_df.Metadata_pert_name.isin(control_barcodes).astype(int)

            result = evaluate_metrics(
                profiles=subset_df,
                features=use_features,
                meta_features=subset_df.filter(regex="Metadata_").columns,
                replicate_groups=pair_config,
                metrics_config=map_config,
                use_copairs=True,
            )["mean_ap"]

            # result = evaluate(
            #     profiles=subset_df.query("Metadata_cell_line == @cell_line"),
            #     features=use_features,
            #     meta_features=[barcode_col, gene_col],
            #     replicate_groups=replicate_group_grit,
            #     operation="grit",
            #     similarity_metric="pearson",
            #     grit_control_perts=control_barcodes,
            result = result.assign(
                cell_line=cell_line,
                barcode_control="cutting_control",
                # cor_method="pearson",
                compartment="all",
                channel=channel,
                feature_group="all",
                num_features=len(channel_features),
                dropped_or_exclusive=dropped_or_exclusive,
            )

            map_channel_results.append(result)

map_channel_results = pd.concat(map_channel_results).reset_index(drop=True)

print(map_channel_results.shape)
map_channel_results.head()


Calculating distances.
Pos pairs size: 21912, Neg pairs size: 184320


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]


Calculating metric: mean_ap


/dgx1nas1/storage/data/akalinin/projects/copairs/src/copairs/compute.py:105: RuntimeWarning: invalid value encountered in divide
  ap_scores = np.add.reduceat(pr_k * rel_k_list, cutoffs) / num_pos


...

/dgx1nas1/storage/data/akalinin/projects/copairs/src/copairs/compute.py:105: RuntimeWarning: invalid value encountered in divide
  ap_scores = np.add.reduceat(pr_k * rel_k_list, cutoffs) / num_pos


  0%|          | 0/6 [00:00<?, ?it/s]


Calculating distances.
Pos pairs size: 21912, Neg pairs size: 184320


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

## Concatenate results together

In [ ]:
full_map_results = pd.concat(
    [
        map_compartment_results,
        map_subcompartment_results,
        map_channel_results,
    ],
    axis="rows",
).reset_index(drop=True)

print(full_map_results.shape)
full_map_results.head()

(17658, 13)


,Metadata_gene_name,Metadata_pert_name,mean_ap,p_value,n_pos_pairs,n_total_pairs,cell_line,barcode_control,compartment,channel,feature_group,num_features,dropped_or_exclusive
0,AKT1,AKT1-1,0.243094,0.084708,11.0,203.0,HCC44,cutting_control,Cells,all,all,100,dropped
1,AKT1,AKT1-2,0.206240,0.101630,11.0,203.0,HCC44,cutting_control,Cells,all,all,100,dropped
2,ARID1B,ARID1B-1,0.122818,0.155451,11.0,203.0,HCC44,cutting_control,Cells,all,all,100,dropped
3,ARID1B,ARID1B-2,0.126775,0.125047,11.0,203.0,HCC44,cutting_control,Cells,all,all,100,dropped
4,ATF4,ATF4-1,0.334947,0.024968,11.0,203.0,HCC44,cutting_control,Cells,all,all,100,dropped


In [ ]:
# Output results
output_dir = "results"
output_file = pathlib.Path(f"{output_dir}/cell_health_map_compartments.tsv.gz")

full_map_results.to_csv(output_file, sep="\t", compression={"method": "gzip", "mtime": 1}, index=False)